In [9]:
from numpy import random
import jieba
import pyodbc 
import pandas as pd

GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence): 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)     

# 判別時間部分整合中文 jieba
def sentence_split(sentence):    
    return(list(jieba.cut(sentence, cut_all=False)))

# 時間日期 intent
WHEN_INPUTS = ("when", "now", "time", "日期", "時間", "何時", "幾點", "時刻")
def ask_when(sentence): 
    from datetime import datetime
    dateTimeObj = datetime.now()
    reverse_ask_when = (False in [sentence.find(k)==-1 for k in WHEN_INPUTS])
    for word in sentence_split(sentence):
        if ( word.lower() in WHEN_INPUTS or reverse_ask_when ):
            return dateTimeObj.strftime("%Y/%m/%d, %H:%M:%S")  
        
        
# 數據庫 intent    
DB_INPUTS = ("databases", "connections", "cpu", 'version')
def ask_db(sentence):
    reverse_ask_db = (False in [sentence.find(k)==-1 for k in DB_INPUTS])
    for word in sentence.split():
            # ##########################################
            # HOME WORK 當是詢問DB時候，想知道哪一個關鍵字
            # #########################################
            conn = pyodbc.connect("DRIVER={SQL Server};SERVER=.;Trusted_Connection=yes")  
            stmt = """
            declare @command nvarchar(30) = '@parameter'
            if @command = 'databases'
                select cast(count(*) as nvarchar(30)) from sys.databases
            else if @command ='connections'       
                select cast(count(*) as nvarchar(30)) from sys.dm_exec_connections
            else if @command = 'version'
                select @@version
            else
                select 'SQL Server'
            
            """
            # 執行上述命令抓取資料
            df = pd.read_sql(stmt.replace('@parameter', word.lower()), conn)     
            return df.iloc[0,0]
        
# verify
ask_when('請問現在時刻') , print(ask_db('version'))



Microsoft SQL Server 2019 (RTM) - 15.0.2000.5 (X64) 
	Sep 24 2019 13:48:23 
	Copyright (C) 2019 Microsoft Corporation
	Developer Edition (64-bit) on Windows 10 Enterprise 10.0 <X64> (Build 17763: )



('2020/03/07, 16:22:25', None)

In [5]:
def response(user_response):
    robo_response=''
    req_result=''
    # 請實作該部分
    if(req_result==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+' 請問我股票資訊 '
        return robo_response

In [10]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=True
            print("ROBO: You are welcome..")
        else:
            # 判斷問候
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            # 判斷時間
            elif (ask_when(user_response)!=None):
                print("ROBO: "+ask_when(user_response))                
            # 查詢資料庫
            elif (ask_db(user_response)!=None):
                print("ROBO: "+ask_db(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                
    else:
        flag=False
        print("ROBO: Bye! take care..")

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
databases
ROBO: 11
請問databases狀況
ROBO: SQL Server
bye
ROBO: Bye! take care..


In [13]:
'databases' in '請問databases狀況'.split()

False